In [1]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep
import pandas as pd
from itertools import chain
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

In [2]:
def extract_script(url):
    possible_scripts = []
    r = urllib.request.urlopen(url)
    parser = html.fromstring(r.read())
    for i in parser.xpath('//*[@id="content"]/p'):
        if (not("Looking for  a great gift idea for the h" in str(i.text_content())) and str(i.text_content()) != "" and len(str(i.text_content())) > 1):
            possible_scripts.append(str(i.text_content()))
    try:
        possible_scripts.append(str(parser.xpath("//pre//text()")))
    except:
        x = 1
    return possible_scripts

def extract_max(a_list):
    try:
        return " ".join(a_list)
    except:
        max_len = ""
        for i in a_list:
            if len(max_len) < len(i):
                max_len = i
        return max_len

In [3]:
response = get("http://www.seinfeldscripts.com/seinfeld-scripts.html")
cleaned_response = response.text.replace('\x00', "")
                                         
parser = html.fromstring(cleaned_response)
                                         
for i in parser.xpath('//table'):
    if i.attrib['width'] == "670":
        table_of_seasons = i  
                                         
tables = table_of_seasons.xpath('//table')

seasons = []
holder = ""
for i in tables[2:]:
    season = {}
    episodes = []
    if len(i.xpath(".//big//font//b")) > 0:
        season["season"] = i.xpath(".//big//font//b")[0].text
        holder = i.xpath(".//big//font//b")[0].text
    else:
        if (i.xpath(".//tr")[0].xpath(".//td")[0].text_content() !=
           "\xa0"):
            season["season"] = holder
            for j in i.xpath(".//tr"):
                episode = {}
                for k in j.xpath(".//td"):
                    episode["name"] = k.text_content()
                for k in j.xpath(".//td//a"):
                    episode["url"] = k.attrib["href"]
                episodes.append(episode)
    for j in i.xpath(".//tr")[1:]:
        episode = {}
        for k in j.xpath(".//td"):
            episode["name"] = k.text_content()
        for k in j.xpath(".//td//a"):
            episode["url"] = k.attrib["href"]
        episodes.append(episode)
    season["episodes"] = episodes
    seasons.append(season)

In [4]:
df = pd.DataFrame(seasons)
df = (df['episodes'].apply(lambda x: pd.Series(x))
            .stack()
            .reset_index(level=1, drop=True)
            .to_frame('episodes')
            .join(df[['season']], how='left'))
df[["name","url"]] = df.episodes.apply(pd.Series)
df = df.drop("episodes",axis=1).reset_index(drop=True)
df = df.drop_duplicates().reset_index(drop=True).dropna()
df["url"] = df["url"].str.strip()
df["url"] = "http://www.seinfeldscripts.com/" + df["url"]
df["season_year"] = df["season"].str.extract("\(([^\)]+)\)")
df["season"] = df["season"].str.extract("^(.*?)\(")
df["name"] = df["name"].str.replace("\n","")
df["name"] = df["name"].str.strip()
df["date"] = df["name"].str.extract("\(([^\)]+)\)$")
df["name"] = df["name"].str.extract("(^(.*?)\(\d\)|.+?(?=\())")
df["script"] = df["url"].apply(extract_script)
df["big_script"] = df["script"].apply(extract_max)

In [5]:
df.to_csv("seinfeld.csv",index=False)

In [6]:
df

,season,name,url,season_year,date,script,big_script
0,Pilot,Good News Bad News,http://www.seinfeldscripts.com/TheSeinfeldChro...,1989,7/5/89,"[JERRY: You know, why we're here? [he means: h...","JERRY: You know, why we're here? [he means: he..."
1,Season 1,The Stakeout,http://www.seinfeldscripts.com/TheStakeout.htm,1990,5/31/90,"[Episode #103 [Series Premiere], Originally A...",Episode #103 [Series Premiere] Originally Air...
2,Season 1,The Robbery,http://www.seinfeldscripts.com/TheRobbery.htm,1990,6/7/90,"[JERRY: So I move into the centre lane, now I ...","JERRY: So I move into the centre lane, now I g..."
3,Season 1,Male Unbonding,http://www.seinfeldscripts.com/MaleUnbonding.htm,1990,6/14/90,"[Written by: Larry David and Jerry Seinfeld, D...",Written by: Larry David and Jerry Seinfeld Dir...
4,Season 1,The Stock Tip,http://www.seinfeldscripts.com/TheStockTip.htm,1990,6/21/90,"[Directed by: Tom Cherones, Broadcasted: June ...",Directed by: Tom Cherones Broadcasted: June 21...
5,Season 2,The Ex-Girlfriend,http://www.seinfeldscripts.com/TheExGirlfriend...,1991,1/23/91,"[Directed by: Tom Cherones, Broadcasted: Janua...",Directed by: Tom Cherones Broadcasted: January...
6,Season 2,The Pony Remark,http://www.seinfeldscripts.com/ThePonyRemark.htm,1991,1/30/91,"[Directed by: Tom Cherones, Broadcasted: Janua...",Directed by: Tom Cherones Broadcasted: January...
7,Season 2,The Jacket,http://www.seinfeldscripts.com/TheJacket.htm,1991,2/6/91,"[Directed by: Tom Cherones, Broadcasted: Febru...",Directed by: Tom Cherones Broadcasted: Februar...
8,Season 2,The Phone Message,http://www.seinfeldscripts.com/ThePhoneMessage...,1991,2/13/91,[The bad thing about television is that everyb...,The bad thing about television is that everybo...
9,Season 2,The Apartment,http://www.seinfeldscripts.com/TheApartment.htm,1991,4/4/91,"[Written by: Peter Mehlman, Directed by: Tom C...",Written by: Peter Mehlman Directed by: Tom Che...
